# COVID-19 Tracking Data Mart: Fact Table

In [1]:
import pandas as pd
import numpy as np

cases = pd.read_csv("conposcovidloc_20201027_20210227_ottawa_toronto.csv")

onset_date_dimension = pd.read_csv("Onset_Date_dimension.csv")
reported_date_dimension = pd.read_csv("Reported_Date_dimension.csv")
test_date_dimension = pd.read_csv("Test_Date_dimension.csv")
specimen_date_dimension = pd.read_csv("Specimen_Date_dimension.csv")
patient_dimension = pd.read_csv("Patient_dimension.csv")
PHU_dimension = pd.read_csv("PHU_dimension.csv")
mobility_dimension = pd.read_csv("mobility_dimension.csv")
special_measures_dimension = pd.read_csv("Restrictions.csv")
weather_dimension = pd.read_csv("weather_dimension.csv")

In [2]:
cases

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,85416,2020-10-23,2020-10-27,2020-10-27,2020-07-24,30s,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,16162,2020-10-23,2020-10-27,2020-10-27,2020-09-25,30s,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
2,143096,2020-10-21,2020-10-27,2020-10-27,2020-10-03,40s,FEMALE,CC,Resolved,No,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
3,83984,2020-10-27,2020-10-27,2020-10-27,2020-10-16,70s,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
4,179091,2020-10-19,2020-10-27,2020-10-27,2020-10-19,40s,FEMALE,MISSING INFORMATION,Resolved,No,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76451,232836,2021-02-26,2021-02-26,2021-03-02,2021-03-01,50s,FEMALE,CC,Not Resolved,No,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
76452,300702,2021-02-26,2021-02-26,2021-03-02,2021-03-01,40s,FEMALE,OB,Not Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
76453,204187,2021-02-19,2021-02-26,2021-03-05,2021-03-04,70s,FEMALE,CC,Resolved,No,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
76454,202101,2021-02-26,2021-02-26,NaN,2021-03-06,50s,FEMALE,CC,Not Resolved,No,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358


In [3]:
# function to test for missing values 

def count_missing(table, col):
    count = 0
    for idx, row in table.iterrows():
        if pd.isna(row[col]):
            count = count + 1
    print(col, count)
    

In [4]:
# checking for missing values in all columns

count_missing(cases, "Accurate_Episode_Date")
count_missing(cases, "Case_Reported_Date")
count_missing(cases, "Test_Reported_Date")
count_missing(cases, "Specimen_Date")
count_missing(cases, "Age_Group")
count_missing(cases, "Client_Gender")
count_missing(cases, "Case_AcquisitionInfo")
count_missing(cases, "Outcome1")
count_missing(cases, "Outbreak_Related")
count_missing(cases, "Reporting_PHU_ID")
count_missing(cases, "Reporting_PHU")
count_missing(cases, "Reporting_PHU_Address")
count_missing(cases, "Reporting_PHU_City")
count_missing(cases, "Reporting_PHU_Postal_Code")
count_missing(cases, "Reporting_PHU_Website")
count_missing(cases, "Reporting_PHU_Latitude")
count_missing(cases, "Reporting_PHU_Longitude")

Accurate_Episode_Date 0
Case_Reported_Date 0
Test_Reported_Date 7649
Specimen_Date 139
Age_Group 0
Client_Gender 0
Case_AcquisitionInfo 0
Outcome1 0
Outbreak_Related 0
Reporting_PHU_ID 0
Reporting_PHU 0
Reporting_PHU_Address 0
Reporting_PHU_City 0
Reporting_PHU_Postal_Code 0
Reporting_PHU_Website 0
Reporting_PHU_Latitude 0
Reporting_PHU_Longitude 0


In [5]:
# fixing missing values for Test and Specimen dates (if missing value, set to Case Reported date)

for idx, row in cases.iterrows():
    if pd.isna(row["Test_Reported_Date"]):
        cases.at[idx, "Test_Reported_Date"] = row["Case_Reported_Date"]   
    if pd.isna(row["Specimen_Date"]):
        cases.at[idx, "Specimen_Date"] = row["Case_Reported_Date"]

In [6]:
# checking all missing values fixed

count_missing(cases, "Test_Reported_Date")
count_missing(cases, "Specimen_Date")

Test_Reported_Date 0
Specimen_Date 0


In [7]:
# create Fact Table data frame

fact_table = pd.DataFrame(columns=['Onset_Date_Surrogate_Key', 'Reported_Date_Surrogate_Key', 'Test_Date_Surrogate_Key', 
                                     'Specimen_Date_Surrogate_Key', 'Patient_Surrogate_Key', 'PHU_Surrogate_Key', 
                                     'Mobility_Surrogate_Key', 'Special_Measures_Surrogate_Key', 'Weather_Surrogate_Key',
                                    'Resolved', 'Unresolved', 'Fatal'])


In [8]:
# fixing city values in mobility dimension for mapping to fact table

mobility_dimension_df =  pd.DataFrame(columns=['Mobility_Surrogate_Key','Mobility_Key', 'Date',
                                    'City', 'Province',"Retail_and_Recreation","Grocery_and_Pharmacy",
                                   'Parks', 'Transit_Stations', 'Workplaces', 'Residential'])

for idx, row in mobility_dimension.iterrows():
    m_row = [idx + 100]
    m_row.append(row["Mobility_key"])
    m_row.append(row['Date'])
    if row['Subregion'] == "Ottawa Division":
        m_row.append("Ottawa")
    else:
        m_row.append("Toronto")
    m_row.append(row['Province'])
    m_row.append(row['Retail_and_recreation'])
    m_row.append(row['Grocery_and_phramacy'])
    m_row.append(row['Parks'])  
    m_row.append(row['Transit_stations']) 
    m_row.append(row['Workplaces'])
    m_row.append(row['Residential'])
    mobility_dimension_df.loc[len(mobility_dimension_df)] = m_row

mobility_dimension_df

,Mobility_Surrogate_Key,Mobility_Key,Date,City,Province,Retail_and_Recreation,Grocery_and_Pharmacy,Parks,Transit_Stations,Workplaces,Residential
0,100,0,2020-10-27,Ottawa,Ontario,-31.0,-5.0,-15.0,-63.0,-53.0,19.0
1,101,1,2020-10-28,Ottawa,Ontario,-31.0,-6.0,-9.0,-63.0,-52.0,19.0
2,102,2,2020-10-29,Ottawa,Ontario,-28.0,-2.0,-21.0,-63.0,-52.0,19.0
3,103,3,2020-10-30,Ottawa,Ontario,-32.0,-3.0,-2.0,-59.0,-48.0,19.0
4,104,4,2020-10-31,Ottawa,Ontario,-26.0,3.0,51.0,-47.0,-6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
243,343,243,2021-02-23,Toronto,Ontario,-53.0,-18.0,-24.0,-66.0,-52.0,21.0
244,344,244,2021-02-24,Toronto,Ontario,-53.0,-20.0,-20.0,-66.0,-52.0,22.0
245,345,245,2021-02-25,Toronto,Ontario,-52.0,-15.0,-11.0,-65.0,-51.0,22.0
246,346,246,2021-02-26,Toronto,Ontario,-50.0,-14.0,-6.0,-63.0,-49.0,21.0


In [9]:
special_measures_dimension

,surrogate_key,city,zone_measures,start_date,end_date,description
0,0,Ottawa,Orange,2020-10-27,2020-12-26,Ottawa is designated as a Restrict/Orange Zone...
1,1,Ottawa,White,2020-12-26,2021-01-12,Province wide lockdown begins
2,2,Ottawa,White,2021-01-12,2021-02-08,Ontario extends province wide lockdown
3,3,Ottawa,White,2021-02-08,2021-02-16,Ottawa lockdown for another week
4,4,Ottawa,Orange,2021-02-16,2021-02-27,"Ottawa stay-at home order ends, back to being ..."
5,5,Toronto,Orange,2020-10-27,2020-10-14,Toronto is designated as a Restrict/Orange Zon...
6,6,Toronto,Red,2020-10-14,2020-10-20,Toronto is declared a red zone(only outdoor di...
7,7,Toronto,Grey,2020-10-23,2020-12-26,"Toronto goes into lockdown, designated as grey..."
8,8,Toronto,White,2020-12-26,2021-01-12,Province wide lockdown begins
9,9,Toronto,White,2021-01-12,2021-02-19,Toronto continues to be in lockdown


In [10]:
# checking for error in special measures dimension

print(special_measures_dimension["city"].unique())

['Ottawa' 'Toronto' 'Toronto ']


In [11]:
# fixing missing end date data values in special measures dimension 
# fixing entry error for city values 

special_measures_dimension_df =  pd.DataFrame(columns=['Special_Measures_Surrogate_Key', 'City',
                                    'Zone_Measures', 'Start_Date','End_Date', 'Description'])

for idx, row in special_measures_dimension.iterrows():
    s_row = [idx]
    if row['city'] == 'Toronto ':
        s_row.append('Toronto')
    else:
        s_row.append(row['city'])
    s_row.append(row['zone_measures'])
    if row['surrogate_key'] == 6:
        s_row.append('2020-11-14')
    elif row['surrogate_key'] == 7:
        s_row.append('2020-11-23')
    else:
        s_row.append(row['start_date'])
    if row['surrogate_key'] == 6:
        s_row.append('2020-11-23')
    elif row['surrogate_key'] == 5:
        s_row.append('2020-11-14')
    else:
        s_row.append(row['end_date'])
    s_row.append(row['description'])
    special_measures_dimension_df.loc[len(special_measures_dimension_df)] = s_row

special_measures_dimension_df

,Special_Measures_Surrogate_Key,City,Zone_Measures,Start_Date,End_Date,Description
0,0,Ottawa,Orange,2020-10-27,2020-12-26,Ottawa is designated as a Restrict/Orange Zone...
1,1,Ottawa,White,2020-12-26,2021-01-12,Province wide lockdown begins
2,2,Ottawa,White,2021-01-12,2021-02-08,Ontario extends province wide lockdown
3,3,Ottawa,White,2021-02-08,2021-02-16,Ottawa lockdown for another week
4,4,Ottawa,Orange,2021-02-16,2021-02-27,"Ottawa stay-at home order ends, back to being ..."
5,5,Toronto,Orange,2020-10-27,2020-11-14,Toronto is designated as a Restrict/Orange Zon...
6,6,Toronto,Red,2020-11-14,2020-11-23,Toronto is declared a red zone(only outdoor di...
7,7,Toronto,Grey,2020-11-23,2020-12-26,"Toronto goes into lockdown, designated as grey..."
8,8,Toronto,White,2020-12-26,2021-01-12,Province wide lockdown begins
9,9,Toronto,White,2021-01-12,2021-02-19,Toronto continues to be in lockdown


In [12]:
# checking error fixed in special measures dimension

print(special_measures_dimension_df["City"].unique())

['Ottawa' 'Toronto']


In [13]:
# check unique values for regions in weather dimension

print(weather_dimension['Station Name'].unique())

['OTTAWA INTL A' 'OTTAWA CDA RCS' 'TORONTO INTL A' 'TORONTO CITY']


In [14]:
# fixing city values in weather dimension for mapping to fact table

weather_dimension_df =  pd.DataFrame(columns=['Weather_Surrogate_Key','City', 'Date',
                                    'Mean Temp (celsius)', 'Min Temp (celsius)','Max Temp (celsius)','Total Precip (mm)'])

for idx, row in weather_dimension.iterrows():
    w_row = [idx]
    if row['Station Name'] in ['OTTAWA INTL A', 'OTTAWA CDA RCS']:
        w_row.append("Ottawa")
    else:
        w_row.append("Toronto")
    w_row.append(row['Date/Time'])
    w_row.append(row['Mean Temp (°C)'])
    w_row.append(row['Min Temp (°C)'])
    w_row.append(row['Max Temp (°C)'])  
    w_row.append(row['Total Precip (mm)']) 
    weather_dimension_df.loc[len(weather_dimension_df)] = w_row

weather_dimension_df

,Weather_Surrogate_Key,City,Date,Mean Temp (celsius),Min Temp (celsius),Max Temp (celsius),Total Precip (mm)
0,0,Ottawa,2020-10-27,0.2,-2.3,2.7,0.0
1,1,Ottawa,2020-10-28,4.2,-1.1,9.4,0.4
2,2,Ottawa,2020-10-29,2.4,-1.3,6.0,0.0
3,3,Ottawa,2020-10-30,-0.5,-4.9,3.9,0.0
4,4,Ottawa,2020-10-31,-1.0,-8.4,6.5,0.0
...,...,...,...,...,...,...,...
241,241,Toronto,2021-02-22,0.6,-1.7,2.8,4.8
242,242,Toronto,2021-02-23,1.8,-0.7,4.3,0.2
243,243,Toronto,2021-02-24,3.9,-0.6,8.4,0.0
244,244,Toronto,2021-02-25,-1.4,-3.7,0.9,0.0


In [15]:
# mapping all dimension surrogate keys and measures to the fact table 


for idx, row in cases.iterrows():

    cases_row = [
        
        onset_date_dimension[onset_date_dimension["Date_Full_Format"]==row["Accurate_Episode_Date"]]["Onset_Date_Surrogate_Key"].values[0],
        reported_date_dimension[onset_date_dimension["Date_Full_Format"]==row["Case_Reported_Date"]]["Reported_Date_Surrogate_Key"].values[0],
        test_date_dimension[test_date_dimension["Date_Full_Format"]==row["Test_Reported_Date"]]["Test_Date_Surrogate_Key"].values[0],
        specimen_date_dimension[specimen_date_dimension["Date_Full_Format"]==row["Specimen_Date"]]["Specimen_Date_Surrogate_Key"].values[0],
                      
        patient_dimension[
            (patient_dimension["Age"]==row["Age_Group"])
            &(patient_dimension["Gender"]==row["Client_Gender"])
            &(patient_dimension["Acquisition_Group"]==row["Case_AcquisitionInfo"])
            &(patient_dimension["Outbreak_Related"]==row["Outbreak_Related"])]["Patient_Surrogate_Key"].values[0],
        
        PHU_dimension[
            (PHU_dimension["City"]==row["Reporting_PHU_City"])]["PHU_Surrogate_Key"].values[0],
    
        mobility_dimension_df[
            (mobility_dimension_df["Date"]==row["Case_Reported_Date"])
            &(mobility_dimension_df["City"]==row["Reporting_PHU_City"])]["Mobility_Surrogate_Key"].values[0],    
    
        special_measures_dimension_df[
            (special_measures_dimension_df["Start_Date"] <= row["Case_Reported_Date"])
            &(special_measures_dimension_df["End_Date"] > row["Case_Reported_Date"])
            &(special_measures_dimension_df["City"]==row["Reporting_PHU_City"])]["Special_Measures_Surrogate_Key"].values[0],

        weather_dimension_df[
            (weather_dimension_df["Date"]==row["Case_Reported_Date"])
            &(weather_dimension_df["City"]==row["Reporting_PHU_City"])]["Weather_Surrogate_Key"].values[0]]
        
    cases_row += [1 if row["Outcome1"] == "Resolved" else 0,
                  1 if row["Outcome1"] == "Not Resolved" else 0,
                  1 if row["Outcome1"] == "Fatal" else 0]
    
    fact_table.loc[len(fact_table)] = cases_row

In [16]:
fact_table

,Onset_Date_Surrogate_Key,Reported_Date_Surrogate_Key,Test_Date_Surrogate_Key,Specimen_Date_Surrogate_Key,Patient_Surrogate_Key,PHU_Surrogate_Key,Mobility_Surrogate_Key,Special_Measures_Surrogate_Key,Weather_Surrogate_Key,Resolved,Unresolved,Fatal
0,314,718,1218,1623,1048,100,224,5,123,1,0,0
1,314,718,1218,1686,1048,100,224,5,123,1,0,0
2,312,718,1218,1694,1005,100,224,5,123,1,0,0
3,318,718,1218,1707,1288,100,224,5,123,1,0,0
4,310,718,1218,1710,1009,100,224,5,123,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
76451,440,840,1344,1843,1101,100,346,10,245,0,1,0
76452,440,840,1344,1843,1000,100,346,10,245,0,1,0
76453,433,840,1347,1846,1293,100,346,10,245,1,0,0
76454,440,840,1340,1848,1101,100,346,10,245,0,1,0


In [17]:
# export fact table

fact_table.to_csv("Fact_Table_Final.csv",index=False)

In [18]:
# export modified data sets 

cases.to_csv("Cases_Final.csv",index=False)
special_measures_dimension_df.to_csv("Special_Measures_Dimension_Final.csv",index=False)